In [62]:
import numpy as np 
import pandas as pd
import sys
from astropy.io import fits
import plotly.graph_objects as go
sys.path.append('../../dash_gaia_orbits/')

# Package imports
from dash_gaia_orbits.dash_test import create_dash_app
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

In [63]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
# limit to only necessary columns
df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars', 'mass_50']]
# rename columns if necessary to be consistent with the package
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

m6 = df_hunt_gagne[df_hunt_gagne['family'] == 'm6']

df_hunt_partIII = pd.read_csv('/Users/cam/Downloads/hunt_part3_clusters.csv')
df_hunt_partIII_to_merge = df_hunt_partIII[['name', 'kind']]
m6 = m6.merge(df_hunt_partIII_to_merge, on='name', how='left')

m6_reference_frame = m6[['x', 'y', 'z', 'U', 'V', 'W']].mean().values
m6_open_clusters = m6.loc[m6['kind'] == 'o']
m6_moving_groups = m6.loc[m6['kind'] == 'm']

In [64]:
# basic custom layout properties
width = 800
height = 800
x_width = 700
y_width = x_width
z_width = 400
z_aspect = 1.*(z_width/x_width)

In [65]:
# define custom layout
        # plotly scene, layout, and camera
camera = dict(
    up=dict(x=0, y=1, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0., y=-.8, z=1.5),
    projection = dict(type = 'perspective')
) 

scene = dict(
    camera = camera,
    aspectmode = 'manual',
    aspectratio = dict(x=1, y=1, z=z_aspect),
    xaxis=dict(
        title='X\' (pc)', 
        range = [-x_width, x_width],
        showgrid=False,
        zeroline=False,
        showline = True,
        mirror = True,
        showspikes = False,
        nticks = 5,
        linecolor = 'white',
        linewidth = 2.,
    ), 
    yaxis=dict(
        title='Y\' (pc)', 
        range = [-y_width, y_width],
        showgrid=False,
        zeroline=False,
        showline = True,
        mirror = True,
        showspikes = False,
        nticks = 5,
        linecolor = 'white',
        linewidth = 2.,
    ), 
    zaxis=dict(
        title='Z\' (pc)', 
        range = [-z_width, z_width],
        showgrid=False,
        zeroline=False,
        showline = True,
        showspikes = False,
        mirror = True,
        nticks = 5,
        linecolor = 'white',
        linewidth = 2.,
    )
)

legend = dict(
    x = 0,
    y = 1,
    title = dict(
        text = 'Click to toggle traces on/off',
        font = dict(
            size = 18,  
            color = 'white'
        ),  
        side = 'top'
    ),
    font = dict(
        size = 14,
        family = 'Arial',
        color = 'white'
    ),
    itemsizing = 'constant',
    bgcolor = 'rgba(0,0,0,0)' # transparent legend
)

layout = dict(
    scene = scene,
    template = 'plotly_dark', 
    dragmode = 'turntable',
    legend = legend,
    width = width,
    height = height
)

In [67]:
save_name = 'm6_3d_plot.html'
plot_3d = StarClusters3DPlotter()
m6_oc_data = StarClusterData(m6_open_clusters, data_name = 'OCs', min_size = 0, max_size = 10, color = 'cyan', opacity = 1)
m6_mg_data = StarClusterData(m6_moving_groups, data_name = 'MGs', min_size = 0, max_size = 5, color = 'cyan', opacity = 1.)
sc_m6 = StarClusterCollection([m6_oc_data, m6_mg_data])
sc_m6.integrate_all_orbits(time=np.arange(0., -61., -1.), reference_frame_center=m6_reference_frame)
fig3d_m6 = plot_3d.generate_3d_plot(collection = sc_m6, show = True, save_name =save_name, figure_layout=layout, static_traces = None, static_traces_times = None, reference_frame_center = m6_reference_frame)